# 1. \_\_init\_\_ 和 \_\_new\_\_ 用法详解

Python中一切皆对象，python类本身也是对象，可以称之为类对象。`对象`=`属性`+`方法`，对象是类的实例，准确地说：实例对象是类对象的实例。


类中的\_\_init\_\_函数用负责当前类实例对象的初始化，也就是在执行\_\_init\_\_之前，实例对象已经存在了。

In [1]:
class Dog(object):
    def __new__(cls, *args, **kwargs):
        print('in the func of Dog.__new__ ')
        # return super(Dog, cls).__new__(cls)
        return object.__new__(cls) # 跟上述注释语句等价
    
    def __init__(self):
        print('in the func of Dog.__init__')
        print(f'self: {self}')
        print(f'self.__class__:{self.__class__}')

d1 = Dog()

in the func of Dog.__new__ 
in the func of Dog.__init__
self: <__main__.Dog object at 0x7f87a15d0b20>
self.__class__:<class '__main__.Dog'>


1. 从上述例子可以看出，当实例化Dog类时，python首先调用类对象的\_\_new\_\_方法，如果该对象没有定义\_\_new\_\_方法，则去父类中查找，知道object类  
2. object.__new__(cls)时调用父类object的\_\_new\_\_()函数，或者通过super(Dog, cls)调用Dog父类（本例子中依然是object）  
3. \_\_new\_\_()需要传递一个参数cls， \_\_init\_\_需要传递参数self, self代表`实例对象`本身，而cls代表`类对象`本身。python中的self相当于C++中的this指针。
4. \_\_new\_\_必须有返回值，返回实例化出来的实例对象(实际上就是self)


通常来说，类开始实例化时，\_\_new\_\_()方法会返回cls的实例，然后该类的\_\_init\_\_方法接受这个实例作为自己的第一个参数（即self）。
如果\_\_new\_\_()没有返回cls(即当前类的实例)，那么当前类的\_\_init\_\_()方法不会被调用，如果\_\_new\_\_()返回了其他类的实例，那么只会调用被返回的那个类的构造方法。
代码如下：

In [2]:
class A(object):
    def __init__(self, *args, **kwargs):
        print("run the init of A")
    def __new__(cls, *args, **kwargs):
        print("run thr new of A")
        return object.__new__(B, *args, **kwargs)

class B(object):
    def __init__(self):
        print("run the init of B")
    def __new__(cls, *args, **kwargs):
        print("run the new of B")
        return object.__new__(cls)

a = A()
print(type(a))
# run thr new of A
# <class '__main__.B'>
b = B()
print(type(b))
# run the new of B
# run the init of B
# <class '__main__.B'>

run thr new of A
<class '__main__.B'>
run the new of B
run the init of B
<class '__main__.B'>


接下来使用 \_\_new\_\_()来实现`单例模式`

1. 什么是单例模式

某些时候一个类应该有且仅有一个实例，例如电脑上只有一个回收站。
确保某个类只有一个实例，自行实例化并向系统提供这个实例，这个类称为单例类，单例模式是一种对象创建型模式

2. 创建单例--确保只有一个对象

In [3]:
class Singleton(object):
    __instance = None

    def __new__(cls, age, name):
        if not cls.__instance:
            cls.__instance = object.__new__(cls)
        return cls.__instance
    
    def __init__(self, age, name):
        self.name = name
        self.age = age
    

bin = Singleton(19, 'Bin')
wen = Singleton(18, 'Wen')

print(f'id(a)==id(b):{id(bin)==id(wen)}')
print(f'bin.age={bin.age}, bin.name={bin.name}')
print(f'wen.age={wen.age}, wen.name={wen.name}')
bin.email = 'bin@163.com'
print(f'wen.eamil:{wen.email}')

id(a)==id(b):True
bin.age=18, bin.name=Wen
wen.age=18, wen.name=Wen
wen.eamil:bin@163.com


上述的本来想创建两个实例，但实际上只有一个，第二次创建实例实际上仅对第一个实例重新初始化参数。
下面我们不需要第二次初始化修改实例参数

In [4]:
class Singleton(object):
    __instance = None
    __init_first = None

    def __new__(cls, age, name):
        if not cls.__instance:
            cls.__instance = object.__new__(cls)
        return cls.__instance
    
    def __init__(self, age, name):
        if not Singleton.__init_first:
            self.name = name
            self.age = age
            Singleton.__init_first = True
    

bin = Singleton(19, 'Bin')
wen = Singleton(18, 'Wen')

print(f'id(a)==id(b):{id(bin)==id(wen)}')
print(f'bin.age={bin.age}, bin.name={bin.name}')
print(f'wen.age={wen.age}, wen.name={wen.name}')
bin.email = 'bin@163.com'
print(f'wen.eamil:{wen.email}')

id(a)==id(b):True
bin.age=19, bin.name=Bin
wen.age=19, wen.name=Bin
wen.eamil:bin@163.com


### 总结 \_\_new\_\_() Vs \_\_init\_\_()

1. \_\_init\_\_()用户对类的实例进行初始化，发生在类实例被创建后，是实例级别方法。
2. \_\_new\_\_()用于控制生成一个新实例的过程，是类级别的方法。
3. \_\_new\_\_()至少有一个cls参数，代表要实例化的类。
4. \_\_new\_\_()必须有返回值，返回实例化出来的实例
5. 将类比作制造商，__new__()方法发就是前期的原材料环节，__init__()方法就是在有了原材料的基础上，加工，初始化商品的环节。

### 参考资料  
[1][知乎-python魔法方法详解](https://zhuanlan.zhihu.com/p/261579683)